# Create custom email notifications for new listings

**Based on user critieras, an email will be sent to your email once a week with the newest listings.**

In [1]:
# Import necessary libraries
import requests
import time
import random
import string
import hashlib
import pandas as pd
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import csv

In [2]:
# Present script info to user
print("This script will enable you to create custom email alerts for new house listings. \nYou need your own CallerId for the Booli-API.")

This script will enable you to create custom email alerts for new house listings. 
You need your own CallerId for the Booli-API.


In [3]:
user_email = input("Enter your email address: ")

with open('users.csv', 'r') as f:
    reader = csv.reader(f)
    user_list = list(reader)
    
user_list_2 = [item for user_list_2 in user_list for item in user_list_2]
    
if user_email in user_list_2:
    print("You already have an alarm config.")
    answer = input("Do you want to reset? y/n: ")
    if answer == 'n':
        print("EXIT")
        exit()
        
    else:
        print("Continue")
        pass
    

print("append to list")
user_list_2.append(user_email)
    
with open('users.csv', 'w') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(user_list_2)


Enter your email address: jockebull@hotmail.com
append to list


In [4]:
# Ask user for API login information

callerId = input("Please enter your Booli-API CallerId: ")
if len(callerId) == 0:
    callerId = input("You did not write anything! Please enter your Booli-API CallerId: ")

user_key = 'vOl0qzQAp1tKtsFvzHXDPpA27ga9ZE5AdJaGiIIX'
#key = input("Please enter your Booli-API private key: ")
#if len(key) == 0:
    #key = input("You did not write anything! Please enter your Booli-API private key: ")

print("Perfect! Now let´s enter some custom search options.")

Please enter your Booli-API CallerId: Rikard Fahlström
Perfect! Now let´s enter some custom search options.


In [5]:
# Ask for custom search options

# Define area
area = input("Which area are you interested in? Type for e.g. Uppsala, Falun, Vasastan. ")
if len(area) == 0:
    area = input("Please enter area of interest: ")

# Define type_of_object    
valid_type_of_objects = ['villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus','kedjehus']    

type_of_object = input("What type of object are you interested in? Enter 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'. ")
if type_of_object in valid_type_of_objects:
    print("Thanks!")
else:
    type_of_object = input("Please enter which type of object you are interested in. Enter one of following; 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'.")

# Define minimum number of rooms
MinRooms = input("Enter minimum number of rooms: ")
if len(MinRooms) == 0:
    MinRooms = input("Please enter minimum number of rooms as single digit.")
elif MinRooms == "0":
    MinRooms = input("0 number of rooms is not a valid object, please enter number of rooms >0: ")
    

# Ask for maximum list price
MaxPrice = input("Enter maximum list price for objects: ")
if len(MaxPrice) == 0:
    MaxPrice = input("Please enter the maximum list price: ")


print("Great, you will now receive a email once per day.")

Which area are you interested in? Type for e.g. Uppsala, Falun, Vasastan. Falun
What type of object are you interested in? Enter 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'. lägenhet
Thanks!
Enter minimum number of rooms: 2
Enter maximum list price for objects: 5000000
Great, you will now receive a email once per day.


In [18]:
# Create API-requests based on user input

# Inputvärden
callerId = "Rikard Fahlström"
time = str(time.time())
unique = str(''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(16)))
key = user_key
hashstr = hashlib.sha1((callerId+time+key+unique).encode('utf-8')).hexdigest()
query_type = 'listings'  # Enter 'listings', 'sold' eller 'areas'
query_city = area
offset = '0' # 
limit = str(10)  # Max antal svar från deras server. Maxgräns = 500
object_type = str(type_of_object)

# Själva API-frågan
r = requests.get("https://api.booli.se/"+
                 query_type+"?q="+query_city+
                 "&callerId="+callerId+
                 "&time="+time+
                 "&unique="+unique+
                 "&hash="+str(hashstr)+
                 "&limit="+limit+
                 "&offset="+offset+
                 "&objectType="+object_type+
                "&minRooms="+MinRooms+
                "&maxListPrice="+MaxPrice)

print("Status code from API request = " + str(r.status_code)+ "\n")

if r.status_code != 200:
    print('fail')
    exit()

output = json.loads(r.text)

print("Number of listings returned in API = " + str(len(output['listings'])) + "\n")
print(output.keys())


AttributeError: 'str' object has no attribute 'time'

In [7]:
urls = []
for value in range(len(output['listings'])):
    urls.append(output['listings'][value]['url'])

#print(type(urls))

In [8]:
object_id = []
for value in range(len(output['listings'])):
    object_id.append(output['listings'][value]['booliId'])

#print(type(object_id))

In [9]:
room_info = []
for value in range(len(output['listings'])):
    room_info.append(output['listings'][value]['rooms'])

#print(room_info)

In [10]:
muni_info = []
for value in range(len(output['listings'])):
    muni_info.append(output['listings'][value]['location']['region']['municipalityName'])

#print(muni_info)

In [11]:
price_info = []
for value in range(len(output['listings'])):
    price_info.append(output['listings'][value]['listPrice'])

#print(type(price_info))

In [12]:
address_info = []
for value in range(len(output['listings'])):
    address_info.append(output['listings'][value]['location']['address']['streetAddress'])

#print(address_info)

In [13]:
published_info = []
for value in range(len(output['listings'])):
    published_info.append(output['listings'][value]['published'])

#print(published_info)

In [14]:

df = pd.DataFrame({
    '1.Published':published_info,
     '3. Area':muni_info,
     '2.Address':address_info,
     '5. # Rooms':room_info,
     '4 .List price':price_info,
    '6. urls':urls
     })

df['1.Published'] = pd.to_datetime(df['1.Published'])
#df['4 .List price'] = '{:,}'.format(df['4 .List price'])


df2 = df.set_index(['1.Published'])
print(df2)
#print(df2.dtypes)

df_html = df2.to_html()

                                2.Address 3. Area  4 .List price  5. # Rooms  \
1.Published                                                                    
2017-06-16 19:20:44    Kungsgårdsvägen 2A   Falun        1150000         3.0   
2017-06-16 03:04:33        Linnévägen 23A   Falun        1825000         2.5   
2017-06-14 18:20:51        Linnévägen 23A   Falun        1595000         2.0   
2017-06-13 19:11:51        Ingels väg 147   Falun        1350000         2.0   
2017-06-11 22:11:48        Sturegatan 75E   Falun        1495000         3.0   
2017-06-10 04:19:35    Drottningvägen 10B   Falun         350000         3.0   
2017-06-03 00:03:27        Linnévägen 29A   Falun        1925000         3.0   
2017-06-02 22:06:42         Hyttgatan 33A   Falun        1385000         3.0   
2017-06-02 21:49:23  Södra Mariegatan 13B   Falun        1295000         2.0   
2017-05-18 01:01:11       Torkils väg 10D   Falun        1695000         3.0   

                                       

In [16]:
# Add df to email

fromaddr = 'rikard.email.notification@gmail.com'
toaddr = 'jockebull@hotmail.com'
subject = 'Custom email notification'
print("steg1")
msg = MIMEMultipart()

msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = subject
print("steg2")
body = 'This is your custom email notification.'
msg.attach(MIMEText(body, 'plain'))

msg1 = MIMEText(df_html,'html')
msg.attach(msg1)
print("steg3")

server = smtplib.SMTP('smtp.gmail.com', 587)
print("test")
server.starttls()
server.login(fromaddr, 'fJBBzWcD4K')
print("steg4")
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

print('Done')

steg1
steg2
steg3
test
steg4
Done
